In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim.optimizer import Optimizer, required

from logger import Logger
from solver import Solver
from crossngover import CrossN

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
net = torchvision.models.mobilenet_v2(pretrained=False)
num_ftrs = net.last_channel

classes = ('plane', 'car', 'bird', 'cat',
        'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

net.classifier = nn.Linear(num_ftrs, len(classes))
net.cuda()
print()

In [13]:
experiment_note = """ 
NET: mobilenet_v2
Expereiment goal: collect the data
Net status: False
Training mode: Normal Evo + Crossn
Optimizer SGD
lr=0.001 
momentum=0.9
epochs= 30
Params: Default"""

logger = Logger('train', experiment_note)

In [14]:
logger.path

'./experiments/2019-11-25/train_21:09'

In [8]:
# post_note = "Test After Experiment info"
# logger.add_post_result(post_note)

In [15]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
evo_optim = CrossN()

def validation(net, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return float(100 * correct / total)

In [16]:
solver = Solver(
    net,
    optimizer,
    logger,
    criterion,
    validation,
    evo_optim, 
    trainloader,
    testloader,
    testloader,
    mode='simple')

In [17]:
solver.start()

Start training
Epoch: 0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
ch_score 10.0
best child - 10.0
Epoch: 1
[2] loss: 2.216 validation score: 18.50 %
Epoch: 2
[3] loss: 2.082 validation score: 24.98 %
Epoch: 3
[4] loss: 1.910 validation score: 30.51 %
Epoch: 4
[5] loss: 1.682 validation score: 34.66 %
Epoch: 5
[6] loss: 1.591 validation score: 37.71 %
Epoch: 6
[7] loss: 1.624 validation score: 40.12 %
Epoch: 7
[8] loss: 1.610 validation score: 42.24 %
Epoch: 8
[9] loss: 1.580 validation score: 43.57 %
Epoch: 9
[10] loss: 1.479 validation score: 44.48 %
Epoch: 10
ch_score 44.42
ch_score 44.42
ch_score 44.41
ch_score 44.42
ch_score 44.42
ch_score 44.43
ch_score 44.41
ch_score 44.42
ch_score 44.42
ch_score 44.43
ch_score 44.43
ch_score 44.42
ch_score 44.42
ch_score 44.41
ch_score 44.

KeyboardInterrupt: 